# Dirac Notation Implementation - DEMO

2024-03-01

## Set up the environment

In [ ]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession()

In [ ]:
from qwhile import *
from diracdec import dirac_bigop_delta_trs, label_trs, juxt
trs = dirac_bigop_delta_trs + label_trs
from qwhile import forward_trs
trs = trs + forward_trs

In [ ]:
sub = parse('''
{
        ket0 : KET('0');
        bra0 : BRA('0');
        ket1 : KET('1');
        bra1 : BRA('1');
        ketP :  "Sqrt[1/2]" SCR (ket0 ADD ket1) ;
        braP :  "Sqrt[1/2]" SCR (bra0 ADD bra1) ;
        ketM :  "Sqrt[1/2]" SCR (ket0 ADD ("-1" MLTK ket1)) ;
        braM :  "Sqrt[1/2]" SCR (bra0 ADD ("-1" MLTB bra1)) ;

        beta00 :  "Sqrt[1/2]" SCR ((ket0 TSRK ket0) ADD (ket1 TSRK ket1));

        I2 : (ket0 OUTER bra0) ADD (ket1 OUTER bra1);

        Z : (ket0 OUTER bra0) ADD ("-1" SCR (ket1 OUTER bra1));

        X : (ket0 OUTER bra1) ADD (ket1 OUTER bra0);

        Y : ("-I" SCR (ket0 OUTER bra1)) ADD ("I" SCR (ket1 OUTER bra0));


        H :  "Sqrt[1/2]" SCR ((ket0 OUTER bra0) ADD (ket0 OUTER bra1) ADD (ket1 OUTER bra0) ADD ("-1" SCR (ket1 OUTER bra1)));

        CX :  ((ket0 TSRK ket0) OUTER (bra0 TSRB bra0))
                    ADD ((ket0 TSRK ket1) OUTER (bra0 TSRB bra1)) 
                    ADD ((ket1 TSRK ket1) OUTER (bra1 TSRB bra0))
                    ADD ((ket1 TSRK ket0) OUTER (bra1 TSRB bra1));

        CZ :  ((ket0 TSRK ket0) OUTER (bra0 TSRB bra0))
                    ADD ((ket0 TSRK ket1) OUTER (bra0 TSRB bra1)) 
                    ADD ((ket1 TSRK ket0) OUTER (bra1 TSRB bra0))
                    ADD ("-1" SCR ((ket1 TSRK ket1) OUTER (bra1 TSRB bra1)));
        }
          ''').get_idempotent()


## Example with Label

In [ ]:
a = parse(''' A[q] MLTOL B[PAIRR(PAIRR(p, q), r)]''')
a.render_tex()

In [ ]:
trs.normalize(a).render_tex()

## Examples

### Operation on Maximally Entangled State

For all $A$,
$$
A_{S} \ket{\Phi}_{S, T} = A^\top_{T} \ket{\Phi}_{S, T}
$$
where
$$
\ket{\Phi} = \sum_{i}\ket{i}\ket{i}
$$

In [ ]:
a = parse(''' 
                (
                    
                   (SUM(i, KET(i) OUTER BRA(i))[S]) MLTOL (A[S])
                ) 
                MLTKL 
                (
                    SUM(i, KET(PAIR(i, i))[PAIRR(S, T)])
                ) 
            ''')

b = parse(''' 
                (
                    (SUM(i, KET(i) OUTER BRA(i))[T]) MLTOL (TP(A)[T])
                ) 
                MLTKL 
                (
                    SUM(i, KET(PAIR(i, i))[PAIRR(S, T)])
                ) 
            ''')

In [ ]:
a.render_tex()

In [ ]:
print(repr(a))

In [ ]:
b.render_tex()

In [ ]:
norm_a = trs.normalize(a)
norm_b = trs.normalize(b)

assert trs.normalize(juxt(norm_a)) == trs.normalize(juxt(norm_b))

In [ ]:
trs.normalize(a).render_tex()

In [ ]:
print(repr(trs.normalize(a)))

In [ ]:
trs.normalize(b).render_tex()

In [ ]:
trs.normalize(a, verbose=True)

## QWhile Forward

### easy example

In [ ]:
cfg = parse('''
    <
    while [2] (KET('0') OUTER BRA('0'))[q] do
             X[q];
    end; , (ket0 OUTER bra0)[q] >
             ''')

In [ ]:
cfg_sub = sub(cfg)
print(cfg_sub)

In [ ]:
trs.normalize(cfg_sub, verbose=True)

### 2

In [ ]:
cfg = parse('''
    <
    while [1] (KET('0') OUTER BRA('0'))[q] do
             H[q];
    end; , (ketP OUTER braP)[q] >
             ''')

In [ ]:
print(cfg)

In [ ]:
cfg_sub = sub(cfg)

In [ ]:
print(cfg_sub)

In [ ]:
trs.normalize(cfg_sub, verbose=True)